In [ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

mnist_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
mnist_data = list(mnist_data)[:4096]



Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!






In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 7)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        #print(x.shape)
        x = self.decoder(x)
        return x

In [ ]:


def train(model, num_epochs=5, batch_size=64, learning_rate=1e-3):
    torch.manual_seed(42)
    criterion = nn.MSELoss() # mean square error loss
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=learning_rate, 
                                 weight_decay=1e-5) # <--
    train_loader = torch.utils.data.DataLoader(mnist_data, 
                                               batch_size=batch_size, 
                                               shuffle=True)
    # print(train_loader.shape)
    outputs = []
    for epoch in range(num_epochs):
        for data in train_loader:
            #print(len(data))
            img, _ = data
            # print(img.shape)
            recon = model(img)
            loss = criterion(recon, img)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
        outputs.append((epoch, img, recon),)
    return outputs



In [ ]:
model = Autoencoder()
max_epochs = 20
outputs = train(model, num_epochs=max_epochs)

Epoch:1, Loss:0.0753
Epoch:2, Loss:0.0662
Epoch:3, Loss:0.0576
Epoch:4, Loss:0.0368
Epoch:5, Loss:0.0251
Epoch:6, Loss:0.0224
Epoch:7, Loss:0.0180
Epoch:8, Loss:0.0147
Epoch:9, Loss:0.0130
Epoch:10, Loss:0.0120
Epoch:11, Loss:0.0131
Epoch:12, Loss:0.0114
Epoch:13, Loss:0.0093
Epoch:14, Loss:0.0098
Epoch:15, Loss:0.0084
Epoch:16, Loss:0.0085
Epoch:17, Loss:0.0075
Epoch:18, Loss:0.0075
Epoch:19, Loss:0.0072
Epoch:20, Loss:0.0070


In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print (x_train.shape)
print (x_test.shape)


(60000, 784)
(10000, 784)


In [ ]:
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(2,activation='relu')(encoded)
#print(encoded.shape)
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

In [ ]:
autoencoder = Model(input_img)

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)   # convert it into 28*28*16
x = MaxPooling2D((2, 2), padding='same')(x) # 14*14*16
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)# 14*14*8
x = MaxPooling2D((2, 2), padding='same')(x) # 7*7*8
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x) # just add another convolution on top of that so, 7*7*8 
encoded = MaxPooling2D((2, 2), padding='same')(x) # 4*4*8

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded) # add first convolution to input so, 4*4*8
x = UpSampling2D((2, 2))(x) # 8*8*8
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x) # add convolution to that, 8*8*8
x = UpSampling2D((2, 2))(x) # 16*16*8
x = Conv2D(16, (3, 3), activation='relu')(x) # apply 3*3 filter on 16 convoltions we get 14*14*8 because of not same padding
x = UpSampling2D((2, 2))(x)# 28*28*8
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x) # 28*28*1

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=5,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 10s 168us/step - loss: 0.2160 - val_loss: 0.1650
Epoch 2/5
60000/60000 [==============================] - 3s 46us/step - loss: 0.1550 - val_loss: 0.1411
Epoch 3/5
60000/60000 [==============================] - 3s 46us/step - loss: 0.1394 - val_loss: 0.1368
Epoch 4/5
60000/60000 [==============================] - 3s 46us/step - loss: 0.1323 - val_loss: 0.1280
Epoch 5/5
60000/60000 [==============================] - 3s 45us/step - loss: 0.1278 - val_loss: 0.1270
